**Necessary packages to make this work**

for the script
-  conda install -c anaconda scipy 
-  conda install -c anaconda pandas 
-  conda install -c anaconda networkx 
-  conda install -c conda-forge matplotlib 
-  conda install -c obob pymatreader 
-  conda install -c anaconda numpy 
-  conda install -c conda-forge opencv 

for nice display in jupyter lab



In [2]:
%matplotlib widget
from util import get_path
import pandas as pd
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
from extract_graph import generate_nx_graph, transform_list, generate_skeleton, generate_nx_graph_from_skeleton, from_connection_tab
from node_id import whole_movement_identification, second_identification
import ast
from plotutil import plot_t_tp1, compress_skeleton
from scipy import sparse
from sparse_util import dilate, zhangSuen
from realign import realign
from datetime import datetime,timedelta
from node_id import orient
import pickle
from matplotlib.widgets import CheckButtons
import scipy.io as sio
import imageio
from pymatreader import read_mat
import os
from matplotlib import colors
from random import choice
from experiment_class import Experiment,clean_exp_with_hyphaes

**First load the dates of interest**

This is some non very interesting code to extract all the folder names and the associated dates in a python understandable way

In [3]:
plate = 13
directory='//sun.amolf.nl/shimizu-data/home-folder/oyartegalvez/Drive_AMFtopology/PRINCE' 
listdir=os.listdir(directory) #This line will run only if the vpn is connected to amolf, if by the time you're trying this you still don't have access to amolf system
#manually find a way to find the dates you're interested in
list_dir_interest=[name for name in listdir if name.split('_')[-1]==f'Plate{0 if plate<10 else ""}{plate}']
ss=[name.split('_')[0] for name in list_dir_interest]
ff=[name.split('_')[1] for name in list_dir_interest]
dates_datetime=[datetime(year=int(ss[i][:4]),month=int(ss[i][4:6]),day=int(ss[i][6:8]),hour=int(ff[i][0:2]),minute=int(ff[i][2:4])) for i in range(len(list_dir_interest))]
dates_datetime.sort()
dates_datetime_chosen=dates_datetime[1:5] #out of all the folders that relate to the plate of interest, we extract only a subset, here the 4 first ones.
dates = [f'{0 if date.month<10 else ""}{date.month}{0 if date.day<10 else ""}{date.day}_{0 if date.hour<10 else ""}{date.hour}{0 if date.minute<10 else ""}{date.minute}' for date in dates_datetime_chosen]

**The experiment objects**

An experiment is associated with a plate so we create the Experiment.

Then we only need to load the date that we are interested in.


In [4]:
exp= Experiment(plate) #Create the object
exp.load(dates,local=False,raw=False) #load the dates

An experiment object has different attributes, some of them we actually don't access that often but I mention them so you know they are there.

In [7]:
print(type(exp.nx_graph),type(exp.nx_graph[0])) #The graphs at each time step, so nx_graph[0] is the graph at timestep 0 (corresponding to dates[0])
print(type(exp.positions),type(exp.positions[0])) #This is not super handy, but the positions are separated from the graph

<class 'list'> <class 'networkx.classes.graph.Graph'>
<class 'list'> <class 'dict'>


So the best way to actually walk through the graph is with the 

In [27]:
exp.plot([0,1,2,3],[[node.label]]*4)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [21]:
node=choice(exp.nodes)
node

Node(1517)

In [23]:
for t in node.ts():
    print(node.degree(t))

3
3
3
3
